In [1]:
import torch
from nlp_fast_unlearning.utils import prepare_dbpedia, build_noisy_dl, ensure_deterministic, DEVICE, BATCH_SIZE
from nlp_fast_unlearning.baseline_model import TextClassificationModel

_, _, _, dbpedia_vocab = prepare_dbpedia(for_baseline_only=True)

vocab_size = dbpedia_vocab.vocab_size

baseline_name = "DBpedia_baseline.pt"
unlearning_model = TextClassificationModel(vocab_size).to(DEVICE)
unlearning_model.load_state_dict(torch.load(baseline_name))

<All keys matched successfully>

In [2]:
%%time

(
    retain_samples,
    noisy_data,
    retain_valid_dl,
    forget_valid_dl,
    retain_test_dl,
    forget_test_dl,
    dbpedia_vocab,
) = prepare_dbpedia(
    for_baseline_only=False,
    classes_to_forget=[10, 11, 12, 13, 14],
    model=unlearning_model,
    retain_percentage=0.01,
    vocab_class=dbpedia_vocab,
)

Searching for error maximizing noise for class  10
Got loss 217.09841918945312 for tensor([103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103], device='cuda:0')
Searching for error maximizing noise for class  11
Got loss 209.3299560546875 for tensor([101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
        101, 101, 1

In [3]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


def validation_step(model, batch):
    labels, text, offsets = batch
    out = model(text,offsets)
    loss = F.cross_entropy(out, labels)   
    acc = accuracy(out, labels)
    return {'Loss': loss.detach(), 'Acc': acc}


def validation_epoch_end(model, outputs):
    batch_losses = [x['Loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   
    batch_accs = [x['Acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      
    return {'Loss': epoch_loss.item(), 'Acc': epoch_acc.item()}

In [4]:
import torch.nn.functional as F

@torch.no_grad()
def evaluate_after_unlearning(model, val_loader):
    model.eval()
    outputs = [validation_step(model, batch) for batch in val_loader]
    return validation_epoch_end(model, outputs)

In [18]:
%%time
from tqdm import tqdm


# lrs = [12]
# clips = [22]
# ratios = [1.5]

# The following search candidates take a few minutes to run through
lrs = [7,8,9,10,11,12,13,14]
clips = [17,18,19,20,21,22,23,24]
ratios = [4,8,12]


best_retain_acc = 0

unlearned_model_name = "DBpedia_fast_unlearned.pt"

for lr in tqdm(lrs):
    for clip in clips:
        for ratio in ratios:
            ensure_deterministic()
            
            noisy_loader = build_noisy_dl(
                retain_samples,
                noisy_data,
                dbpedia_vocab,
                retain_to_forget_ratio=ratio,
            )
            unlearning_model = TextClassificationModel(vocab_size).to(DEVICE)
            unlearning_model.load_state_dict(torch.load(baseline_name))

            optimizer = torch.optim.SGD(unlearning_model.parameters(), lr = lr)


            unlearning_model.train(True)
            for epoch in range(1):
                running_loss = 0.0
                running_acc = 0
                num_batches = len(noisy_loader)
                
                for i, data in enumerate(noisy_loader):
                    labels, inputs, offsets = data

                    optimizer.zero_grad()
                    outputs = unlearning_model(inputs,offsets)
                    loss = unlearning_model.criterion(outputs, labels)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(unlearning_model.parameters(), clip)
                    optimizer.step()

                    running_loss += loss.item()
                    out = torch.argmax(outputs.detach(),dim=1)
                    running_acc += (labels==out).sum().item()/labels.size(0)
                print(f"Train loss {epoch+1}: {running_loss/num_batches},Train Acc:{running_acc*100/num_batches}%")
                print("Performance of Standard Forget Model on Forget Class")
                history = [evaluate_after_unlearning(unlearning_model, forget_valid_dl)]
                print("Accuracy: {}".format(history[0]["Acc"]*100))
                print("Loss: {}".format(history[0]["Loss"]))

                print("Performance of Standard Forget Model on Retain Class")
                history = [evaluate_after_unlearning(unlearning_model, retain_valid_dl)]
                print("Accuracy: {}".format(history[0]["Acc"]*100))
                print("Loss: {}".format(history[0]["Loss"]))
                forget_acc = evaluate_after_unlearning(unlearning_model, forget_valid_dl)["Acc"]*100
                if forget_acc == 0.0:
                    retain_acc = evaluate_after_unlearning(unlearning_model, retain_valid_dl)["Acc"]*100
                    if retain_acc > best_retain_acc:
                        best_retain_acc = retain_acc
                        best_lr = lr
                        best_clip = clip
                        best_ratio = ratio
                        torch.save(unlearning_model.state_dict(), unlearned_model_name)

print(
    f"Best hyperparams: LR={best_lr}, grad clip={best_clip}, "
    f"ratio={best_ratio} | Best retain acc: {best_retain_acc}"
)
unlearning_model = TextClassificationModel(vocab_size).to(DEVICE)
unlearning_model.load_state_dict(torch.load(unlearned_model_name))

  0%|                                                                           | 0/8 [00:00<?, ?it/s]

Train loss 1: 1030.4310541817497,Train Acc:56.741431451612904%
Performance of Standard Forget Model on Forget Class
Accuracy: 14.603720605373383
Loss: 40.03542709350586
Performance of Standard Forget Model on Retain Class
Accuracy: 90.41445255279541
Loss: 2.6900699138641357
Train loss 1: 610.0576876293529,Train Acc:62.548424586776854%
Performance of Standard Forget Model on Forget Class
Accuracy: 12.449189275503159
Loss: 34.84757614135742
Performance of Standard Forget Model on Retain Class
Accuracy: 80.76491951942444
Loss: 3.8820762634277344
Train loss 1: 384.22266006469727,Train Acc:73.46739614456446%
Performance of Standard Forget Model on Forget Class
Accuracy: 19.180862605571747
Loss: 21.69687271118164
Performance of Standard Forget Model on Retain Class
Accuracy: 87.11174726486206
Loss: 5.915497779846191
Train loss 1: 1184.8233123727803,Train Acc:55.63256048387097%
Performance of Standard Forget Model on Forget Class
Accuracy: 15.055876970291138
Loss: 23.373981475830078
Performan

 12%|████████▍                                                          | 1/8 [01:09<08:03, 69.03s/it]

Train loss 1: 1277.1730982603565,Train Acc:58.59915034562212%
Performance of Standard Forget Model on Forget Class
Accuracy: 37.07266449928284
Loss: 28.735427856445312
Performance of Standard Forget Model on Retain Class
Accuracy: 77.8555154800415
Loss: 2.719019889831543
Train loss 1: 905.1896093975414,Train Acc:53.72062241735537%
Performance of Standard Forget Model on Forget Class
Accuracy: 20.985300838947296
Loss: 76.41410064697266
Performance of Standard Forget Model on Retain Class
Accuracy: 95.92437744140625
Loss: 0.45405784249305725
Train loss 1: 511.57462581835296,Train Acc:69.40403427373019%
Performance of Standard Forget Model on Forget Class
Accuracy: 16.33223593235016
Loss: 37.91170883178711
Performance of Standard Forget Model on Retain Class
Accuracy: 93.03171634674072
Loss: 0.41489988565444946
Train loss 1: 1443.1893314084698,Train Acc:57.51548099078341%
Performance of Standard Forget Model on Forget Class
Accuracy: 24.045832455158234
Loss: 24.182592391967773
Performance

 25%|████████████████▊                                                  | 2/8 [02:10<06:28, 64.76s/it]

...

 88%|██████████████████████████████████████████████████████████▋        | 7/8 [07:43<01:05, 65.51s/it]

Train loss 1: 3970.5425999010763,Train Acc:37.5810051843318%
Performance of Standard Forget Model on Forget Class
Accuracy: 10.847626626491547
Loss: 337.9039001464844
Performance of Standard Forget Model on Retain Class
Accuracy: 5.82510270178318
Loss: 146.4703369140625
Train loss 1: 3226.011701410467,Train Acc:26.412383780991732%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.33203125931322575
Loss: 237.41514587402344
Performance of Standard Forget Model on Retain Class
Accuracy: 55.76603412628174
Loss: 27.063329696655273
Train loss 1: 1920.540557208814,Train Acc:36.77243737071937%
Performance of Standard Forget Model on Forget Class
Accuracy: 0.06835937383584678
Loss: 389.8408203125
Performance of Standard Forget Model on Retain Class
Accuracy: 49.36717450618744
Loss: 52.65946578979492
Train loss 1: 4612.472620279504,Train Acc:47.127016129032256%
Performance of Standard Forget Model on Forget Class
Accuracy: 18.28242540359497
Loss: 53.417877197265625
Performance of 

100%|███████████████████████████████████████████████████████████████████| 8/8 [08:47<00:00, 65.90s/it]


Best hyperparams: LR=11, grad clip=19, ratio=12 | Best retain acc: 88.00011873245239
CPU times: user 8min 27s, sys: 19.6 s, total: 8min 46s
Wall time: 8min 47s


<All keys matched successfully>

In [22]:
print("Performance of Standard Forget Model on Forget Class")
history = [evaluate_after_unlearning(unlearning_model, forget_valid_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

print("Performance of Standard Forget Model on Retain Class")
history = [evaluate_after_unlearning(unlearning_model, retain_valid_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

Performance of Standard Forget Model on Forget Class
Accuracy: 0.0
Loss: 256.3982849121094
Performance of Standard Forget Model on Retain Class
Accuracy: 87.99634575843811
Loss: 11.496419906616211


In [23]:
print("Test on Forget Class")
history = [evaluate_after_unlearning(unlearning_model, forget_test_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

print("Test on Retain Class")
history = [evaluate_after_unlearning(unlearning_model, retain_test_dl)]
print("Accuracy: {}".format(history[0]["Acc"]*100))
print("Loss: {}".format(history[0]["Loss"]))

Test on Forget Class
Accuracy: 0.01195790828205645
Loss: 258.3566589355469
Test on Retain Class
Accuracy: 87.88360953330994
Loss: 11.101592063903809
